# Excersize

Finish MNIST kaggle chalange using CNN.

https://www.kaggle.com/c/digit-recognizer

Description

## Data Preprocessing

In [8]:
import numpy as np
import pandas as pd
import keras

In [3]:
img_rows, img_cols = 28, 28
num_classes = 10
num_epochs=12

def data_prep(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

train_file = "MNIST_train.csv"
raw_data = pd.read_csv(train_file)

x, y = data_prep(raw_data)

In [4]:
raw_data.shape

(42000, 785)

## Train Model

In [6]:
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D





model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), strides=2, activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=128,
          epochs=8,
          validation_split = 0.2)

Train on 33600 samples, validate on 8400 samples
Epoch 1/8
33600/33600 [==============================] - 48s 1ms/step - loss: 0.3561 - acc: 0.8929 - val_loss: 0.0906 - val_acc: 0.9713
Epoch 2/8
33600/33600 [==============================] - 46s 1ms/step - loss: 0.1189 - acc: 0.9638 - val_loss: 0.0636 - val_acc: 0.9795
Epoch 3/8
33600/33600 [==============================] - 46s 1ms/step - loss: 0.0867 - acc: 0.9730 - val_loss: 0.0533 - val_acc: 0.9833
Epoch 4/8
33600/33600 [==============================] - 45s 1ms/step - loss: 0.0664 - acc: 0.9790 - val_loss: 0.0441 - val_acc: 0.9851
Epoch 5/8
33600/33600 [==============================] - 45s 1ms/step - loss: 0.0542 - acc: 0.9834 - val_loss: 0.0524 - val_acc: 0.9830
Epoch 6/8
33600/33600 [==============================] - 45s 1ms/step - loss: 0.0464 - acc: 0.9848 - val_loss: 0.0461 - val_acc: 0.9868
Epoch 7/8
33600/33600 [==============================] - 45s 1ms/step - loss: 0.0425 - acc: 0.9866 - val_loss: 0.0443 - val_acc: 0.9864

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
Total para

In [23]:
model.save('MNIST.h5')

In [24]:
raw_x = pd.read_csv('MNIST_test.csv')

In [25]:

num_images = raw_x.shape[0]
x_test_array = raw_x.values[:]
x_test_shaped_array = x_test_array.reshape(num_images, img_rows, img_cols, 1)
x_test = x_test_shaped_array / 255
y_test = model.predict(x_test)

In [26]:
score = model.evaluate(x_test, y_test, batch_size=16)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

28000/28000 [==============================] - 15s 546us/step
Test loss: 0.02498527174397272
Test accuracy: 1.0


In [27]:
y_cat = y_test.argmax(axis=-1)

In [28]:
len(y_cat)

28000

In [29]:
submission = pd.DataFrame(y_cat, columns=['Label'], index=range(1, y_cat.shape[0]+1))

In [30]:
submission.reset_index(inplace=True)

In [31]:
submission.columns = ['ImageId', 'Label']

In [32]:
submission.to_csv('MNIST_submisison.csv', header=True, index=False)